# DNBLab Jupyter Notebook Tutorial 

## Titelanalyse, TOC-Volltextsuche und Ergebnisvisualisierung

Dieses DNBLab Tutorial besteht aus mehreren Teilen und dient als Einführung in die Titelanalyse sowie die Volltextsuche mit Python. Die Titelsuchen werden dabei anhand bibliothekarischer Metadaten durchgeführt, die im Format MARC21-XML vorliegen. Diese können bspw. in Form verschiedener Datensets im [DNBLab](https://www.dnb.de/dnblab) oder als Gesamtabzüge unter https://www.dnb.de/gesamtabzuege heruntergeladen und genutzt werden. Die Volltextsuchen werden anschließend anhand digitalisierter Inhaltsverzeichnisse (kurz TOC für "table of content") bearbeitet. Im letzten Teil des Tutorials werden die Ergebnisse dieser Analysen visualisiert.     

Das Tutorial wird exemplarisch anhand eines Szenarios durchgeführt, in dem wir uns für Hochschulschriften zum Thema "Reform" interessieren, die in der Sachgruppe "Wirtschaft" publiziert wurden. Ausgangsbasis für die Analyse ist das auf der DNBLab-Seite verfügbare Set "Freie Online-Hochschulschriften". Da die Datei allerdings sehr groß ist, wurde für dieses Tutorial ein spezielles Datenset erstellt, welches sich direkt auf die bibliographischen Einträge der freien Online-Hochschulschriften beschränkt, die sich in der Sachgruppe "330 Wirtschaft" befinden und im Zeitraum zwischen den Jahren 2000 und 2010 erschienen sind.   



**Inhaltsverzeichnis:** 

* [Einrichten der Arbeitsumgebung](#Teil1) 
* [Tutorial 1 - Titelanalyse - Teil 1](#Teil2)
* [Tutorial 2 - Titelanalyse - Teil 2 (Daten in ein Dataframe überführen)](DNB_Titelanalyse_MARC_Tutorial_Teil2.ipynb)
* [Tutorial 2 - Volltextsuche]  - in Arbeit
* [Tutorial 3 - Ergebnisvisualisierung]  - in Arbeit



### 1. Einrichten der Arbeitsumgebung - Let's get started  <a class="anchor" id="Teil1"></a>

Um die Arbeitsumgebung für die folgenden Schritte passend einzurichten, sollten zunächst die benötigten Python-Biblitoheken importiert werden. Wir laden dazu ElementTree (als ET), um unser MARC21-XML analysieren zu können sowie Pandas (als pd) zur Datenmanipulation und -analyse.:  

In [1]:
# Laden benötigter Python-Bibliotheken
import xml.etree.ElementTree as ET
import pandas as pd


Im nächsten Schritt laden wir die MARC21-XML-Datei, die wir im folgenden analysieren wollen, in ElementTree und suchen zunächst nach dem Rootverzeichnis. Da uns außerdem interessiert, wieviele verschiedenene Einträge sich überhaupt in der XML-Datei befinden, lassen wir uns dies anzeigen:    


In [2]:
tree = ET.parse('dataset_tutorial_eco.xml')  # Laden der MARC-Datei in ElementTree 
root = tree.getroot()                           # Laden des Root-Verzeichnisses des XML

records = list(root)
total_records = len(records)  

#Ausgabe der Gesamtzahl der in der Datei vorhandenen Records:
print("Gesamtzahl bibliographischer Einträge: ",total_records)


Gesamtzahl bibliographischer Einträge:  2828


Wir wissen nun also, wieviele Titeleinträge sich insgesamt in der Datei befinden und haben schon einmal alles für die weitere Analyse vorbereitet. 

### 2. Ausgabe von Titeln, die ein bestimmtes Suchwort enthalten: 

Die für unsere Fragestellung interessanten Daten befinden sich in der XML-Datei als Inhalt zwischen bestimmten Tags. Wir brauchen also zunächst eine Möglichkeit, diesen Inhalt in Textform aus dem XML zu "holen". 

Da wir zwar nur in den Titeltexten nach unserem Schlüsselwort suchen wollen, als Ergebnis aber möglichst nicht nur den Titel selbst, sondern auch Angaben wir Autor, Publikationsdatum, ISBN etc. wünschen, schreiben wir uns eine allgemeine Funktion für unser XML, um an den Inhalt der unterschiedlichen Felder gelangen zu können. 


Zur Erinnerung: Unser XML ist folgendermaßen aufgabaut: 
```
<?xml version="1.0" encoding="UTF-8"?><collection xmlns="http://www.loc.gov/MARC21/slim">
    <record type="Bibliographic">
        <leader> ... </leader>
        <controlfield> ... </controlfield>
        ...
        <controlfield> ... </controlfield>
        <datafield tag="123" ... >
            <subfield code="x"> ... </subfield>
            ...
            <subfield code="x"> ... </subfield>
        </datafield>
        <datafield tag="456" ...> 
        .... 
```         
Für die Lokalisierung der gewünschen Informationen müssen wir nun natürlich wissen, in welchen Feldern sich welche Informationen befinden -> dies kann bei Bedarf in der MARC-Dokumentation (https://www.dnb.de/marc21 oder direkt in der [Feldbeschreibung](https://www.dnb.de/SharedDocs/Downloads/DE/Professionell/Metadatendienste/Exportformate/MARC21/marcFeldbeschreibungTitelExcel202102.zip;jsessionid=66325567A5E2D903B7F0E05465D2654D.internet562?__blob=publicationFile&v=2)) nachgeschaut werden. 

Anhand der tag-Information innerhalb der Datafield-tags wird also das entsprechende Feld identifiziert: Das &lt;datafield&gt; mit dem tag="245" steht dabei für den Titeleintrag, im Unterfeld "a" steht dabei der Haupttitel, in "b" der Untitel, sofern vorhanden und im Unterfeld "c" wird der oder die für die Publikation Verantwortliche angegeben - dabei kann es sich sowohl um eine Person als auch eine Körperschaft handeln. 
    
Die für uns relevanten Informationen befinden sich also zumeist zwischen bestimmten &lt;subfields&gt;-Tags, die ihrerseits wiederrum anhand der "tag"-Information innerhalb des jeweils übergeordneten &lt;datafield&gt;-Tags identifiziert werden können. In unserem Fall wäre dies beispielhaft für den Haupttitel das &lt;datafield&gt; mit dem tag 245, innerhalb dessen wir den Haupttitel im &lt;subfields&gt; mit dem code "a" finden.
    
Wir schreiben uns deshalb eine Funktion, die uns den Inhalt aus den entsprechenden &lt;subfields&gt; holt und nennen sie entsprechend "get_subfield_text". Die Funktion wird mehrere Parameter brauchen, um die unterschiedlichen Felder anzusprechen. Da wir diese allgemein halten wollen, nennen wir die Variablen, die später die jeweiligen Parameter übergeben bekommen, "element", "tag" und "subfield":    
          

In [3]:
def get_subfield_text(element, tag, subfield):
    if element.attrib['tag'] == tag:
            for subelement in element:
                if subelement.attrib['code'] == subfield:
                    return subelement.text
    return None


Diese Funktion bekommt später als "element" eine passende XML-node übergeben. Als "tag" wollen wir das jeweiilige MARC-Tag übergeben, welches spezifieren wird, wonach wir genau suchen: Um nach einem Titel zu suchen, wollen wir also bspw. alle Datafield-Tags nach dem "tag" "245" durchsuchen. Wenn wir dagegen nach Autoren suchen wollen, ändern wir die Variable "tag" entsprechend auf "100" usw. Die Variable "subfield" steht, wie der Name sagt, für das jeweilige Subfield, in dem sich die Information befindet: Für unser Titelbeispiel also das Subfield "a" - würden wir hier nur den Untertitel extrahieren wollen, würden wir dagegen im Subfield "b" suchen. 

Mit dieser Funktion haben wir also die Möglichkeit, je nach Bedarf den Text aus den unterschiedlichen Subfields ausgeben zu lassen. :) 


Im weiteren Verlauf legen wir nun ein Keyword fest, nachdem wir suchen möchten. Danach schreiben wir eine Schleife, die durch alle entsprechenden XML-nodes läuft und dabei unsere Funktion aufruft und den Titeltext übergibt. Daraufhin fragen wir, ob unser Keyword im Titeltext enthalten ist. Wenn dies der Fall ist, schreiben wir den Titel in eine Liste namens "Titelsammlung", die wir zuvor definiert haben und erweitern diese Liste, wenn ein weiterer Treffer gefunden wird: 


In [4]:
keyword = "Reform"      # Hier legen wir unser Suchwort fest - dieses kann natürlich geändert werden
titelsammlung = []      # Definition unser Listenvarialbe für die Titelsammlung 


for record in records:  
        
    for child in record.findall('{http://www.loc.gov/MARC21/slim}datafield'):             
        title = get_subfield_text(child, "245", "a") # Aufruf der oben geschriebenen Funktion, Übergabe der Parameter
        if title:
            if keyword in title:
                titelsammlung.append(title)
                


Um zu sehen, welche Titel unser Code nun gefunden hat, gibt es verschiedene Möglichkeiten: Die einfachste ist sicherlich, einfach unsere Liste über den print-Befehl anzeigen zu lassen: 

In [5]:
print(titelsammlung)

['\x98Die\x9c Armutsfestigkeit des deutschen Vorsorgesystems unter besonderer Berücksichtigung möglicher Reformierungsvorschläge der gesetzlichen Rentenversicherung', 'Institutionen und Reformen', 'Haushaltskonsolidierungen und Reformprozesse', 'Reforms and foreign direct investment', '\x98Die\x9c Gewerkschaftskrise und der Reformprozess des Gewerkschaftssystems in Südkorea', 'Reformaufgaben im Rahmen einer Neuordnung der Finanzierung des öffentlichen Personennahverkehrs (ÖPNV)', 'Indiens wirtschaftspolitische Reformen nach 1991: eine Fallstudie des Stromsektors', 'Ärztenetzwerke als Reformansatz für den ambulanten Sektor', 'Vergleichende Analyse der Reformen im Bereich der Alterssicherung in Westeuropa und in Mittel- und Osteuropa', 'Zur Reform des Internationalen Währungsfonds (IWF)', 'Indiens wirtschaftspolitische Reformen nach 1991', '\x98Die\x9c Reform der EU-Strukturfonds von 1999', 'Reform der gemeinsamen Agrarpolitik und EU-Integration Polens']


Alternativ können wir aber bspw. auch ein Dataframe erstellen lassen: Diese Option bietet uns die Programmbibliothek "Pandas" für Python, die wir beim Einrichten unserer Arbeitsumgebung bereits importiert haben - "Pandas" selbst steht dabei für die "Python Data Analysis Library". Ein Pandas-Dataframe kann man sich am besten wie eine Tabelle bzw. einfache Datenbank vorstellen. 

Wir erstellen dazu nun exemplarisch ein Set, welches wir "result" nennen. Da wir bislang nur mit einer Ausgabe-Variable arbeiten, wäre dies noch nicht zwingend nötig, wird aber im Anschluss sinnvoll sein. In ein Set können Objekte beliebigen Datentyps  gespeichert werden. Dieses Set überführen wir im Anschluss in ein Dataframe und lassen uns zuletzt mit dem Befehl "df.head()" die ersten 5 Zeilen des Dataframe anzeigen: 


In [6]:
result = {'title' : titelsammlung}
df = pd.DataFrame(result)

#Gibt die ersten 5 Zeilen des Dataframes aus (zur Kontrolle):
df.head() 


,title
0,Die Armutsfestigkeit des deutschen Vorsorges...
1,Institutionen und Reformen
2,Haushaltskonsolidierungen und Reformprozesse
3,Reforms and foreign direct investment
4,Die Gewerkschaftskrise und der Reformprozess...


Alternativ können wir uns auch die letzten 5 Zeilen anzeigen lassen oder das Dataframe komplett als CSV exportieren. Um den Export vorzunehmen, sollte im folgenden Code-Snippet allerdings der Befehl df.tail() "auskommentiert" werden (also einfach eine # davorsetzen, so dass der Code als Kommentar behandelt und nicht ausgeführt wird). Im Gegenzug muss die # vor der letzten Zeile "df.to_csv" natürlich entfernt werden:  

In [7]:
df.tail() # Zeigt die letzten 5 Zeilen des Dataframes an

#Exportieren des gesamten Dataframe als CSV-Datei: 
#df.to_csv("Titeltest.csv", index=False) 

,title
8,Vergleichende Analyse der Reformen im Bereich ...
9,Zur Reform des Internationalen Währungsfonds ...
10,Indiens wirtschaftspolitische Reformen nach 1991
11,Die Reform der EU-Strukturfonds von 1999
12,Reform der gemeinsamen Agrarpolitik und EU-Int...


Je nachdem, in welcher Umgebung dieses Tutorial ausgeführt wird, ändert sich bei Ausgabe der CSV-Datei der Ablageort: Wird dieses Tutorial bspw. lokal unter Windows ausgeführt, wird die Datei auf oberster Ebene des aktuellen Benutzer-Verzeichnisses abgelegt. Wird das Tutorial online über Binder ausgeführt, wird die Datei auch dort abgelegt und kann heruntergeladen werden.

Auf diese Art können wir also innerhalb einer MARC21-XML Datei nach einem von uns gewählten Schlüsselwort in den Titelangaben suchen. Außerdem können wir den Code anpassen, um nach anderen Begriffen oder Namen in beliebigen MARC-Feldern im XML zu suchen. Die Ergebnisse können wir uns im Anschluss anzeigen lassen oder, nach Belieben, in eine Variable speichern und als Datei ausgeben lassen. 

Da unser Ziel allerdings eine Ausgabe mehrerer bibliographischer Angaben zu einem Titel ist, der nicht nur ein bestimmtes Suchwort ("Klimawandel") beinhaltet, sondern außerdem der Sachgruppe "Wirtschaft" zugeordnet wurde, müssen wir im folgenden noch etwas tiefer einsteigen. 

[Hier geht es mit der Titelanalyse weiter](DNB_Titelanalyse_MARC_Tutorial_Teil2.ipynb)